In [ ]:
#!pip install konlpy
#!pip install git+https://github.com/kakaobrain/pororo.git
#!pip install python-mecab-ko
#!pip install pykrx

In [ ]:
from pororo import Pororo

In [ ]:
from pykrx import stock
import pandas as pd

KOSDAQ = stock.get_market_ticker_list(market="KOSDAQ")
KOSPI = stock.get_market_ticker_list(market="KOSPI")

def return_name(market):
    Company = []

    for ticker in market:
        Value =stock.get_market_ticker_name(ticker)
        Company.append([Value,ticker])
 
    return Company

In [ ]:
from bs4 import BeautifulSoup
import requests

bullet_summ = Pororo(task="text_summarization", lang="ko", model="bullet")
abs_summ = Pororo(task="text_summarization", lang="ko", model="abstractive")

def summerize_news(url, Company):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    body = soup.find_all(id='articleBodyContents')
    if len(body) > 0:
        text = body[0].text
        output_bullet = bullet_summ(text, 
                                    beam=5, 
                                    len_penalty=0.6,
                                    no_repeat_ngram_size=3,
                                    top_k=50,
                                    top_p=0.7)
        summary_output = abs_summ(text, 
                                  beam=5, 
                                  len_penalty=0.6,
                                  no_repeat_ngram_size=3,
                                  top_k=50,
                                  top_p=0.7
                                  )
        summ = "[주요] \n"
        for line in output_bullet:
          summ += "-"+line+"\n"
        summ += "[요약] \n"
        summ += summary_output+"\n"
        target = ""
        for c in Company :
          if c[0] in summary_output :
            if len(target) < len(c) :
              target = c
        summ += "[회사정보] \n"
        if target=="" :
          summ += "null \n"
        else :
          summ += target+"\n"

        return summ
    else:
        return "error"


In [ ]:
import time
import re
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta

# 오늘 날짜 형식화
now=datetime.now()
now=str(now)
year=now[0:4]
month=now[5:7]
day=now[8:10]
res=year+month+day

# url file에 저장 (max_page_num 바꿔주면 됨)
output_file_name=str(res)+"_url.txt"
output_file = open(output_file_name, "w", encoding="utf-8")
page_num = 1
max_page_num = 1

user_agent = "'Mozilla/5.0"
headers ={"User-Agent" : user_agent}

urls = []
while page_num<=max_page_num:
    page_url = "http://news.naver.com/main/list.nhn?sid2=258&sid1=101&mid=shm&mode=LS2D&date=" + str(res) + "&page=" + str(page_num) + ""
    response = requests.get(page_url, headers=headers)
    html = response.text
    # url 추출
    url_frags = re.findall('<a href="(.*?)"',html)
    for url_frag in url_frags:
        if "sid1=101&sid2=258" in url_frag and "aid" in url_frag:
            urls.append(url_frag)
    page_num+=1
result=set(urls)
urls=list(result)
for url in urls:
    print(url, file=output_file)
time.sleep(2)
output_file.close()

# url 바탕으로 요약문 생성
# 요약문 file에 저장(테스트용으로 count로 6개만 끊어준것임)
output_file_name2=str(res)+"_summarized.txt"
output_file2 = open(output_file_name2, "w", encoding="utf-8")
count=0
for url in urls:
  if(count>5):
    break
  count=count+1
  print(summerize_news(url,return_name(KOSPI)), file=output_file2)
output_file2.close()
'''
# mount it
from google.colab import drive
drive.mount('/content/drive')
# copy it there
!cp "20220408_summarized.txt" /content/drive/MyDrive
!cp "20220408_url.txt" /content/drive/MyDrive
'''


'\n# mount it\nfrom google.colab import drive\ndrive.mount(\'/content/drive\')\n# copy it there\n!cp "20220408_summarized.txt" /content/drive/MyDrive\n!cp "20220408_url.txt" /content/drive/MyDrive\n'